In [1]:
def Cppred(b=1,h=1,p=4,facenum=1,ML='svr'):
    
    #necessary imports
    
    import numpy as np
    import pandas as pd
    
    #import to read matlab files
    
    from pymatreader import read_mat
    import os.path
    
    #merging the relevant files
    
    dataframe = pd.Series([],dtype=float)
    
    #loop to merge all files of similar configuration
    
    for m in range(2):
        for n in range(10):
            for k in range(6):
                filename = "T{}1{}_{}_{}{}{}_1.mat".format(b,h,p,m,n,k)
                if os.path.isfile(filename) == True:
                    df = read_mat(filename)
                    avgspeed= df['Uh_AverageWindSpeed']
                    location = pd.DataFrame(df['Location_of_measured_points']).T
                    cp  = pd.DataFrame(df['Wind_pressure_coefficients'])
                    meancp = cp.mean()
                    dat = pd.concat([location,meancp],axis=1,ignore_index=True)
                    a = m*100+n*10+k
                    angle = pd.Series([a]*(len(dat)))
                    meandat = pd.concat([dat,angle],axis=1,ignore_index=True)
                    dataframe = pd.concat([dataframe,meandat],axis=0)
    
    dataframe = dataframe.reset_index()
    
    #grouping the data by required face
    
    faces = dataframe.groupby(dataframe[3])
    dfs = {}
    for name, group in faces:
        dfs[name] = group
    face = dfs[facenum]
    face.reset_index(drop=True,inplace=True)
    face.drop('index',axis=1,inplace = True)

    #preparing dataset for training
    
    traindat = face[[0,1,5,4]]
    traindat.dtypes
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import r2_score, mean_squared_error
    X = traindat[[0,1,5]]
    y = traindat[4]
    
    #splitting dataset into test and train sets with random sampling
    
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1, random_state=42)

    #Running support vector regressor ML algorithm
    
    from sklearn.svm import SVR
    if ML == 'svr':
        model = SVR(kernel='rbf', C=1, gamma=0.01)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        print(f'R-squared: {r2:.5f}')
        print(f'MSE: {mse:.5f}')
    
    #Running Gradient Boosting regressor ML algorithm
    
    elif ML == 'gbr':
        from sklearn.ensemble import GradientBoostingRegressor
        model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=8, random_state=42)
        model.fit(X_train,y_train)
        y_pred=model.predict
        r2 = r2_score(y_test,y_pred)
        mse= mean_squared_error(y_test,y_pred)
        print(f'R-squared: {r2:.2f}')
        print(f'MSE: {mse:.2f}')
        
        
    #Running the simple NN algorithm
    
    elif ML == 'NN':
        from sklearn.neural_network import MLPClassifier
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Create a neural network model with one hidden layer
        model = MLPClassifier(hidden_layer_sizes=(32,), activation='relu', solver='adam', max_iter=1000, random_state=42)

        # Train the model on the training set
        model.fit(X_train, y_train)

        # Evaluate the model on the test set
        accuracy = model.score(X_test, y_test)
        print(f"Test accuracy: {accuracy:.2f}")
        
    a = input("Do you want the original contour? ans with y/n: ")
    ang = input("please enter the angle for the contour: ")
    if a == "y":
        
        #creating a contour
        filename1 = "T{}1{}_{}_{}_1.mat".format(b,h,p,ang)
        if os.path.isfile(filename1) == True:
            df1 = read_mat(filename1)
            avgspeed1= df1['Uh_AverageWindSpeed']
            location1 = pd.DataFrame(df1['Location_of_measured_points']).T
            cp1  = pd.DataFrame(df1['Wind_pressure_coefficients'])
            meancp1 = cp1.mean()
            dat1 = pd.concat([location1,meancp1],axis=1,ignore_index=True)
            faces = dat1.groupby(dat1[3])
            dfs = {}
            for name, group in faces:
                dfs[name] = group
            face1 = dfs[facenum]
            face1.reset_index(drop=True,inplace=True)
            cont = face1[[0,1,4]].copy()
            cont.rename(columns={0: "x", 1: "y",4:"cpm"},inplace=True)
            Z = cont.pivot_table(index='x', columns='y', values='cpm').T.values
            X_unique = np.sort(cont.x.unique())
            Y_unique = np.sort(cont.y.unique())
            import matplotlib.pyplot as plt
            fig, ax = plt.subplots(figsize=(2*b, 2*h))
            ax.set_aspect('equal')
            plt.contourf(X_unique,Y_unique,Z,cmap='autumn')
            
            
            

In [ ]:
Cppred(1,1,facenum=1)

R-squared: 0.13151
MSE: 0.04473
